In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

In [2]:
import pandas as pd
train = pd.read_csv("data/train_og.csv")
test = pd.read_csv("data/test_og.csv")
train.columns

Index(['ID', 'TargetID', 'DRUGID', 'DRUGTYPE', 'Drug_high_status', 'DRUGNAME',
       'PUBCHCID', 'Disease_of_highest_status', 'Drug_Status', 'UNIPROID',
       'TARGNAME', 'GENENAME', 'SYNONYMS', 'FUNCTION', 'BIOCLASS', 'SEQUENCE',
       'Disease', 'Accession Number', 'Target_Status'],
      dtype='object')

# Keeping important columns only

In [3]:
cols = ["Drug_high_status", "Disease_of_highest_status", "Drug_Status",
              "GENENAME", "FUNCTION", "BIOCLASS", "SEQUENCE", "Disease"]
X = train.loc[:, cols]
y = train.loc[:, "Target_Status"]
test = test.loc[:, cols]

# Ordinal Encoding on Drug_high_status and Drug_status

In [4]:
import pandas as pd

# Define the mapping for the statuses including "Registered"
def commoniser(text):
    if text in ['Withdrawn from market', 'Discontinued in Phase 4', 'Phase 4', 'Phase 4 Trial']:
        return 'Phase 4'
    elif text in ['Approved (orphan drug)', 'approved', 'NDA filed']:
        return 'Approved'
    elif text in ['Phase 3', 'Discontinued in Phase 3',]:
        return 'Phase 3'
    elif text in ['Phase 2', 'Phase 2 Trial', 'Phase 2a', 'Phase 2b', 'Phase 2/3',]:
     return 'Phase 2'
    elif text in ['Discontinued in Phase 2', 'Discontinued in Phase 2a', 'Discontinued in Phase 2b']:
     return 'Phase 2 Disc'
    elif text in ['Phase 1', 'Phase 1 Trial', 'Phase 1/2',]:
        return 'Phase 1'
    elif text in ['Discontinued in Phase 1', 'Discontinued in Phase 1/2',]:
        return 'Phase 1 Disc'
    elif text in ['Investigative', 'Preclinical', 'Clinical trial', 'Terminated', "Application submitted"]:
        return 'Pre-phase 1'
    elif text in ['Discontinued in Preregistration', 'Patented', 'Registered']:
        return 'pre-pre-fail'
    else:
       return text

def customEncoder(text):
    if text == 'Phase 4':
        return 8 
    elif text == 'Approved':
        return 7 
    elif text == 'Phase 3':
        return 6 
    elif text == "Phase 2":
        return 5 
    elif text == 'Phase 2 Disc':
        return 4 
    elif text == 'Phase 1':
        return 3 
    elif text == "Phase 1 Disc":
        return 2 
    elif text == "Pre-phase 1":
        return 1 
    elif text == 'pre-pre-fail':
        return 0 

In [5]:
X["Drug_high_status"] = X["Drug_high_status"].apply(commoniser)
X["Drug_Status"] = X["Drug_Status"].apply(commoniser)
test["Drug_high_status"] = test["Drug_high_status"].apply(commoniser)
test["Drug_Status"] = test["Drug_Status"].apply(commoniser)

In [6]:
X["Drug_high_status"] = X["Drug_high_status"].apply(customEncoder)
X["Drug_Status"] = X["Drug_Status"].apply(customEncoder)
test["Drug_high_status"] = test["Drug_high_status"].apply(customEncoder)
test["Drug_Status"] = test["Drug_Status"].apply(customEncoder)

# Preprocessing BIOCLASS and FUNCTION for Embeddings 
## (we eventually did make use of NLP, but not the greatest results)

In [7]:
import spacy
nlp = spacy.load('en_core_web_sm')

def preprocess_text(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return ' '.join(tokens)

/home/raj/miniconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
X["BIOCLASS"] = X["BIOCLASS"].apply(preprocess_text)
X["FUNCTION"] = X["FUNCTION"].apply(preprocess_text)
test["BIOCLASS"] = test["BIOCLASS"].apply(preprocess_text)
test["FUNCTION"] = test["FUNCTION"].apply(preprocess_text)

# Converting sequence

In [8]:
acids = {'C', 'G', 'S', 'H', 'V', 'W', 'L', 'T', 'P', 'F', 'Y', 'R', 'N', 'I', 'D', 'E', 'Q', 'A', 'M', 'K'}
def sequence_converter(text):
    val = 0
    for letter in acids:
        val += text.count(letter) * ord(letter)
        return val

In [9]:
X["Sequence_encoded"] = X["SEQUENCE"].apply(sequence_converter)
test["Sequence_encoded"] = test["SEQUENCE"].apply(sequence_converter)

In [10]:
X["SEQUENCE"] = X["SEQUENCE"].apply(len)
test["SEQUENCE"] = test["SEQUENCE"].apply(len)

### Encoding categorical variables

In [3]:
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

X = df.drop("Target_Status", axis = 1)
y = df["Target_Status"]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

### F1 score function

In [5]:
def calculate_f1_score(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    train_preds = model.predict(X_train)
    test_preds = model.predict(X_test)
    f1_score_train = f1_score(y_train, train_preds, average='weighted')
    f1_score_test = f1_score(y_test, test_preds, average='weighted')
    return f1_score_train, f1_score_test

## CatBoost Classifier

In [7]:
from catboost import CatBoostClassifier

In [8]:
cat_model = CatBoostClassifier(verbose=1, random_seed=42, n_estimators=600, thread_count=-1,
                               custom_metric='F1', depth=10, learning_rate=0.1, 
                               task_type='GPU', devices='0')
cat_f1_score_train, cat_f1_score_test = calculate_f1_score(cat_model, X_train, y_train, X_test, y_test)

0:	learn: 2.9872549	total: 804ms	remaining: 8m 1s
1:	learn: 2.7405795	total: 1.47s	remaining: 7m 18s
2:	learn: 2.5384250	total: 2.13s	remaining: 7m 2s
3:	learn: 2.3840003	total: 2.78s	remaining: 6m 54s
4:	learn: 2.2712213	total: 3.45s	remaining: 6m 50s
5:	learn: 2.1697041	total: 4.1s	remaining: 6m 45s
6:	learn: 2.0923712	total: 4.78s	remaining: 6m 45s
7:	learn: 2.0193976	total: 5.47s	remaining: 6m 44s
8:	learn: 1.9558640	total: 6.14s	remaining: 6m 43s
9:	learn: 1.8953290	total: 6.8s	remaining: 6m 41s
10:	learn: 1.8556309	total: 7.46s	remaining: 6m 39s
11:	learn: 1.8101625	total: 8.16s	remaining: 6m 39s
12:	learn: 1.7769586	total: 8.87s	remaining: 6m 40s
13:	learn: 1.7349363	total: 9.55s	remaining: 6m 39s
14:	learn: 1.6968624	total: 10.2s	remaining: 6m 39s
15:	learn: 1.6627928	total: 11.1s	remaining: 6m 45s
16:	learn: 1.6318879	total: 12.1s	remaining: 6m 54s
17:	learn: 1.6110119	total: 13.1s	remaining: 7m 2s
18:	learn: 1.5827689	total: 14s	remaining: 7m 8s
19:	learn: 1.5514282	total: 15

In [9]:
print(f'Train CatBoost F1 Score: {cat_f1_score_train}')
print(f'Test CatBoost F1 Score: {cat_f1_score_test}')


Train CatBoost F1 Score: 0.9840754125765618
Test CatBoost F1 Score: 0.9751638083714935


In [8]:
cat_model = CatBoostClassifier(verbose=1, random_seed=42, n_estimators=900, thread_count=-1,
                               custom_metric='F1', depth=10, learning_rate=0.1,
                               task_type='GPU', devices='0')
cat_f1_score_train, cat_f1_score_test = calculate_f1_score(cat_model, X_train, y_train, X_test, y_test)

0:	learn: 2.9872549	total: 769ms	remaining: 11m 31s
1:	learn: 2.7405795	total: 1.42s	remaining: 10m 38s
2:	learn: 2.5384250	total: 2.04s	remaining: 10m 11s
3:	learn: 2.3840003	total: 2.67s	remaining: 9m 57s
4:	learn: 2.2712213	total: 3.28s	remaining: 9m 47s
5:	learn: 2.1697041	total: 3.91s	remaining: 9m 42s
6:	learn: 2.0923712	total: 4.53s	remaining: 9m 37s
7:	learn: 2.0193976	total: 5.15s	remaining: 9m 33s
8:	learn: 1.9558640	total: 5.77s	remaining: 9m 31s
9:	learn: 1.8953290	total: 6.41s	remaining: 9m 30s
10:	learn: 1.8556309	total: 7.02s	remaining: 9m 27s
11:	learn: 1.8101625	total: 7.63s	remaining: 9m 24s
12:	learn: 1.7769586	total: 8.26s	remaining: 9m 23s
13:	learn: 1.7349363	total: 8.89s	remaining: 9m 22s
14:	learn: 1.6968624	total: 9.55s	remaining: 9m 23s
15:	learn: 1.6627928	total: 10.2s	remaining: 9m 22s
16:	learn: 1.6318879	total: 10.8s	remaining: 9m 21s
17:	learn: 1.6110119	total: 11.4s	remaining: 9m 19s
18:	learn: 1.5827689	total: 12s	remaining: 9m 17s
19:	learn: 1.5514282	

In [9]:
print(f'Train CatBoost F1 Score: {cat_f1_score_train}')
print(f'Test CatBoost F1 Score: {cat_f1_score_test}')

Train CatBoost F1 Score: 0.9894950179935368
Test CatBoost F1 Score: 0.9808438954730127


## Inferencing

In [10]:
import pandas as pd
import numpy as np

test = pd.read_csv('data/test.csv')

In [11]:
# Encoding categorical variables
for column in test.select_dtypes(include=['object']).columns:
    if column in label_encoders:
        le = label_encoders[column]
       
        unseen_labels = set(test[column]) - set(le.classes_)
        if unseen_labels:
            print(unseen_labels)
            
            le.classes_ = np.append(le.classes_, list(unseen_labels))
        test[column] = le.transform(test[column])

{'Dermatitis herpetiformis'}
{'USP1', 'NPY4R', 'PRKCI', 'PDK2', 'MGMT', 'RXFP1', 'Bact ileS', 'CENPE', 'SETD7', 'LSS'}
{'repair methylate nucleobase dna stoichiometrically transfer methyl group cysteine residue enzyme suicide reaction enzyme irreversibly inactivate involve cellular defense biological effect methylguanine meg methylthymine meet dna', 'necessary bcr abl oncogene mediate resistance apoptotic drug leukemia cell protect leukemia cell drug induce apoptosis cultured neuron prevent amyloid beta protein induce apoptosis interrupt cell death process early step glioblastoma cell function downstream phosphatidylinositol kinase promotion cell survival phosphorylate inhibit pro apoptotic factor bad form protein complex non small cell lung cancer nsclc cell regulate oncogenic activity phosphorylation turn promote transform growth invasion response nerve growth factor ngf act downstream src phosphorylate activate allow subsequent activation nf kappa b neuronal cell survival function o

In [12]:
test_predictions = cat_model.predict(test)

In [13]:
predictions = label_encoders['Target_Status'].inverse_transform(test_predictions)

d:\Code\Nexus\.conda\Lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [14]:
df_for_ID = pd.read_csv('data/test_og.csv')

In [15]:
output_df = pd.DataFrame({
    'ID': df_for_ID['ID'],
    'Target_Status': predictions
})

In [16]:
output_df.head(10)

,ID,Target_Status
0,1076,Approved
1,190816,Phase 3
2,180551,Phase 3
3,51630,Approved
4,50566,Phase 1/2
5,148985,Phase 2
6,71409,Phase 2
7,18091,Investigative
8,39903,Phase 1/2
9,178779,Patented


## .975 score

In [16]:
output_df.to_csv('cat_predictions.csv', index=False)

## .981 score (late submission)

In [17]:
output_df.to_csv('cat900_predictions.csv', index=False)